# Environment

### Imports

In [2]:
import gym
from gym import Env, spaces
import numpy as np

# Utilidad
import json
import copy # Para copiar las variables originales a las que el ambiente edite para cuando se haga un reset

### Data Example

#### Estudiantes

In [53]:
estudiantes_json = [
    {
        "nombre": "Javier Ahumada",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Santiago Lopez",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            3,
            1,
            2
        ]
    }
]

#### Desafíos

In [54]:
desafios_json = [
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    }
]

# desafios = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]

### Funciones útiles

### Definición

In [34]:
class Estudiante(object):
    def __init__(self, id, preferencias = []):
        self.id = id
        self.preferencias = preferencias # IDs de desafíos

class Desafio(object):
    def __init__(self, id, estudiantes : list = []):
        self.id = id
        self.estudiantes = estudiantes # IDs de estudiantes

    def push(self, estudiante):
        self.estudiantes.append(estudiante)

In [59]:
class OrganizationEnvironment(Env):
    def __init__(self, estudiantes_json, desafios_json):
        super(OrganizationEnvironment, self).__init__()
        self.num_estudiantes = len(estudiantes_json)
        self.num_desafios = len(desafios_json)
        self.estudiantes_json = estudiantes_json
        self.desafios_json = desafios_json

        # Espacio de observación
        self.reset()
        self.observation_space = spaces.MultiBinary(len(self.state))
        #TODO Cambiar espacio multibinario a multidiscreto
        #TODO Determinar restricciones blandas y absolutas
        #TODO Maskable: Invalid action masking

        # Definición de espacio de acción
        print(self.num_estudiantes, self.num_desafios)
        self.action_space = spaces.MultiDiscrete([self.num_estudiantes, self.num_desafios])


    def dict_to_binary(self, input_dict):
        # Convertir el diccionario a una cadena JSON
        json_str = json.dumps(input_dict)
        # Convertir la cadena JSON a bytes
        json_bytes = json_str.encode('utf-8')
        # Convertir los bytes a su representación en 1's y 0's
        bit_array = [int(bit) for byte in json_bytes for bit in format(byte, '08b')]
        return bit_array

    def render(self):
        print(self.json_state)

    def reset(self):
        estudiantes = [] # [[1,2,3],[2,3,4],...]
        for estudiante in self.estudiantes_json:
            estudiantes.append(estudiante["preferencias"])

        self.json_state = {
            "desafios" : [[ 0 for _ in range(self.num_estudiantes)] for _ in range(self.num_desafios)],
            "estudiantes" : estudiantes
        }


        self.state = self.dict_to_binary(self.json_state)
        # print("self.json_state", self.json_state)
        # print("self.state", self.state)

        return self.state

    def step(self, action):
        assert self.action_space.contains(action), "Invalid Action"

        id_estudiante = action[0]
        id_desafio = action[1]

        reward = 0
        if (self.check_possible(action)):
            self.json_state["desafios"][id_desafio][id_estudiante] = 1
            suma_estudiantes_dentro_desafio = sum(self.json_state["desafios"][id_desafio]) # Suma de los estudiantes dentro del desafío seleccionado
            if(suma_estudiantes_dentro_desafio > 3):
                reward = -20
            else:
                reward += suma_estudiantes_dentro_desafio
            try:
                esta_en_sus_prioridades = len(self.json_state["estudiantes"][id_estudiante]) - self.json_state["estudiantes"][id_estudiante].index(id_desafio)
                reward += esta_en_sus_prioridades
            except ValueError:
                reward += -5

        else:
            reward = -20

        self.state = self.dict_to_binary(self.json_state)

        observation = self.state
        done = self.check_done()
        info = {}

        return observation, reward, done, info

    def check_possible(self, action):
        # Verificar si estudiante ya tiene desafío asignado
        contador_de_instancias = 0
        for desafio in self.json_state["desafios"]:
            contador_de_instancias += desafio[action[0]]
        if(contador_de_instancias > 0):
            return False
        else:
            return True

    def get_reward(self):
        # Implementar función de reward
        return 1

    def check_done(self):
        # Condición para finalizar el episodio
        contador_de_estudiantes = 0
        for desafio in self.json_state["desafios"]:
            for estudiante in desafio:
                contador_de_estudiantes += estudiante

        if(contador_de_estudiantes == self.num_estudiantes):
            return True

        return False

env = OrganizationEnvironment(estudiantes_json, desafios_json)


5 4


In [58]:
env.reset()
while True:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

    print(reward)

    if done == True:
        break

# print(binary_to_dict2(obs))

3
-5
4
5
-20
-20
-20
-20
-20
-20
-20
-20
5


In [60]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env



model = PPO("MlpPolicy", env).learn(total_timesteps=50_000)

obs = env.reset()

while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)

    print(reward, env.json_state)

    if done == True:
        break

# print(binary_to_dict(obs))

/home/javo/.local/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


3 {'desafios': [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'estudiantes': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [3, 1, 2]]}
2 {'desafios': [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0]], 'estudiantes': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [3, 1, 2]]}
-4 {'desafios': [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0]], 'estudiantes': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [3, 1, 2]]}
-20 {'desafios': [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0]], 'estudiantes': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [3, 1, 2]]}
-20 {'desafios': [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0]], 'estudiantes': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [3, 1, 2]]}
-20 {'desafios': [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0]], 'estudiantes': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [3, 1, 2]]}
4 {'desafios': [[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [

In [44]:
print(obs)

def binary_to_dict2(bit_array):
    # Convertir el arreglo de bits en una cadena de bytes
    byte_array = []
    for i in range(0, len(bit_array), 8):
        byte_str = ''.join(str(bit) for bit in bit_array[i:i+8])
        byte_array.append(int(byte_str, 2))
    # Convertir la lista de bytes a un objeto bytes
    json_bytes = bytes(byte_array)
    # Convertir los bytes a una cadena JSON
    json_str = json_bytes.decode('utf-8')
    # Convertir la cadena JSON a un diccionario
    output_dict = json.loads(json_str)
    return output_dict

print(binary_to_dict2(obs))

[0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 

In [26]:
import json
import struct


def dict_to_binary(input_dict):
    # Convertir el diccionario a una cadena JSON
    json_str = json.dumps(input_dict)
    # Convertir la cadena JSON a bytes
    json_bytes = json_str.encode('utf-8')
    # Convertir los bytes a su representación en 1's y 0's
    binary_data = ''.join(format(byte, '08b') for byte in json_bytes)
    return binary_data

def binary_to_dict(binary_data):
    # Dividir la cadena binaria en grupos de 8 bits (1 byte)
    byte_array = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    # Convertir los grupos de 8 bits a bytes
    json_bytes = bytes([int(byte, 2) for byte in byte_array])
    # Convertir los bytes a una cadena JSON
    json_str = json_bytes.decode('utf-8')
    # Convertir la cadena JSON a un diccionario
    output_dict = json.loads(json_str)
    return output_dict


# Diccionario original
# Diccionario original
data = {"nombre": "Juan", "edad": 350, "ciudad": "Madrid"}

# Convertir diccionario a binario (1's y 0's)
binary_data = dict_to_binary(data)
print(f"Datos en binario (1's y 0's): {len(binary_data)}")

# Convertir binario (1's y 0's) de vuelta a diccionario
restored_data = binary_to_dict(binary_data)
print(f"Datos restaurados: {restored_data}")


Datos en binario (1's y 0's): 408
Datos restaurados: {'nombre': 'Juan', 'edad': 350, 'ciudad': 'Madrid'}
